In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
from decision_tree import DT
from classsification_gd import ClassificationGD
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score,auc,f1_score,precision_score
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets
from sklearn.utils import resample
from knn import KNN

import matplotlib.pyplot as plt
import time




In [2]:
def update_class(X,main,other):
    result = []

    for i in range(len(X)):
        result.append(1 if X[i] == main else other)
        
    return np.array(result)

def to_multi_label(Y,clases):

    Y_result = []

    for class_ in Y:
        encode_for_elem = [0 for _ in range(len(clases.keys()))]
        encode_for_elem[clases[class_]] = 1
        Y_result.append(encode_for_elem)
    

    return np.array(Y_result)


class multi_svm:
    def __init__(self,alpha,epochs,c=10,batch_size = 30):
        self.models = []
        self.clases = []
        self.alpha = alpha
        self.epochs = epochs
        self.c = c
        self.batch_size = batch_size

   
    def train(self,X_train,Y_train,X_test,Y_test,clases):
        self.models = []
        self.clases = clases 

        for _ in range(len(self.clases)):
            self.models.append(ClassificationGD(self.alpha,self.epochs,self.c,'svm'))
 

        for i in range(len(self.clases)):
            class_ = clases[i]
            Y_train_ = update_class(Y_train,class_,-1)
            Y_test_ = update_class(Y_test,class_,-1)

            self.models[i].train(X_train,Y_train_,X_test,Y_test_,self.batch_size)
    
    def predict(self,X):


        predictions = []


        for point in X:

            scores = {self.clases[i]:self.models[i].svm_raw_predict(point) for i in range(len(self.models))}
            # print(scores)
            
            predictions.append(max(scores,key=scores.get))

        


        return np.array(predictions) 


In [3]:
class multi_logistic:
    def __init__(self,alpha,epochs,batch_size=30):
        self.models = []
        self.clases = []
        self.alpha = alpha
        self.epochs = epochs
        self.batch_size = batch_size

   
    def train(self,X_train,Y_train,X_test,Y_test,clases):
        self.models = []
        self.clases = clases 

        for _ in range(len(self.clases)):
            self.models.append(ClassificationGD(self.alpha,self.epochs))
 

        for i in range(len(self.clases)):
            class_ = clases[i]
            Y_train_ = update_class(Y_train,class_,0)
            Y_test_ = update_class(Y_test,class_,0)

            self.models[i].train(X_train,Y_train_,X_test,Y_test_,batch_size=self.batch_size)
    
    def predict(self,X):


        predictions = []


        for point in X:

            scores = {self.clases[i]:self.models[i].logistic_h(point) for i in range(len(self.models))}
            
            # print(scores)
            predictions.append(max(scores,key=scores.get))

        


        return np.array(predictions) 

In [4]:
def k_folds(X,Y,k,model):

    clases = list({elem for elem in Y})
    clases_map = {clases[i]:i for i in range(len(clases))}
        
    n_in_fold = len(Y)//k

    precision_ = []
    recall_ = []
    f1_score_ = []
    auc = []
    total_time = 0



    for i in range(k):
        print(i)

        X_train = np.concatenate([X[:i*n_in_fold],X[(i+1)*n_in_fold:]])
        Y_train = np.concatenate([Y[:i*n_in_fold],Y[(i+1)*n_in_fold:]])

        # print(X_train.shape)
        # print(Y_train.shape)


        X_test = X[i*n_in_fold:(i+1)*n_in_fold]
        Y_test = Y[i*n_in_fold:(i+1)*n_in_fold]


        # print(Y_test.shape)

        # print()
        # print()
        # print()
        initial_training_time = time.time()
        model.train(X_train,Y_train,X_test,Y_test,clases)
        training_time = time.time() - initial_training_time
        total_time += training_time 

        print(f'Para el fold {i} se demoro {training_time} para entrenar')

        pred = model.predict(X_test)
        one_hot_data_pred = to_multi_label(pred,clases_map)

        one_hot_real_data = to_multi_label(Y_test,clases_map)


        # print(confusion_matrix(Y_train,pred))
        partial_precision = precision_score(one_hot_real_data,one_hot_data_pred,average=None)
        partial_recall = recall_score(one_hot_real_data,one_hot_data_pred,average=None)
        partial_f1 = f1_score(one_hot_real_data,one_hot_data_pred,average=None)


        print(partial_precision,partial_recall,partial_f1)

        precision_.append(partial_precision)
        recall_.append(partial_recall)
        f1_score_.append(partial_f1)
        # print(auc(Y_test,predicted_data))
    precision_ = np.sum(np.array(precision_),axis=0)/k
    recall_ = np.sum(np.array(recall_),axis=0)/k
    f1_score_ = np.sum(np.array(f1_score_),axis=0)/k


    print(f'Tiempo total: {total_time}')
    print(precision_,recall_,f1_score_)


        




In [5]:
def boostrap(X,Y,k,model):

    clases = list({elem for elem in Y})
    clases_map = {clases[i]:i for i in range(len(clases))}
        
    n_in_fold = len(Y)//k

    precision_ = []
    recall_ = []
    f1_score_ = []
    auc = []
    total_time = 0


    random_states = [42 + i for i in range(k)]



    for i in range(k):
        print(i)


        # while len(Y_test) == 0:

        mask_for_training = np.array([0 for i in range(len(Y))] )


        train_indexes = resample(range(len(Y)),n_samples=(len(Y)),replace=True,random_state=random_states[i])

        mask_for_training[train_indexes] = 1
        mask_for_testing = np.ones(len(Y)) - mask_for_training

        test_indexes = [i for i in range(len(Y)) if mask_for_testing[i]]
        # print(np.sum(mask_for_training))
        # print(np.sum(mask_for_testing))

        
        X_train = X[train_indexes]
        Y_train = Y[train_indexes]


        X_test = X[test_indexes]
        Y_test = Y[test_indexes]


        if len(Y_test) == 0:
            print(f'Para la seed {i} todo el testing es vacio. Continuando...')
            continue




        

        # print(Y_train.shape)
        # print(Y_test.shape)


        # print()
        # print()
        # print()
        initial_training_time = time.time()
        model.train(X_train,Y_train,X_test,Y_test,clases)
        training_time = time.time() - initial_training_time
        total_time += training_time 

        print(f'Para el boostrap {i} se demoro {training_time} para entrenar')

        pred = model.predict(X_test)
        one_hot_data_pred = to_multi_label(pred,clases_map)

        one_hot_real_data = to_multi_label(Y_test,clases_map)


        # print(confusion_matrix(Y_train,pred))
        partial_precision = precision_score(one_hot_real_data,one_hot_data_pred,average=None)
        partial_recall = recall_score(one_hot_real_data,one_hot_data_pred,average=None)
        partial_f1 = f1_score(one_hot_real_data,one_hot_data_pred,average=None)
        print(partial_precision,partial_recall,partial_f1)


        precision_.append(partial_precision)
        recall_.append(partial_recall)
        f1_score_.append(partial_f1)
        # print(auc(Y_test,predicted_data))
    precision_ = np.sum(np.array(precision_),axis=0)/k
    recall_ = np.sum(np.array(recall_),axis=0)/k
    f1_score_ = np.sum(np.array(f1_score_),axis=0)/k


    print(f'Tiempo total: {total_time}')
    print(precision_,recall_,f1_score_)


        





In [6]:

scaler = MinMaxScaler()




data = pd.read_csv('./proyect_dataset/training.csv')
data[data.columns[:-1]] = scaler.fit_transform(data[data.columns[:-1]])
# data[data.columns[-1]] = update_class(data[data.columns[-1]],1,-1)
# data = data.to_numpy()

print(list({elem for elem in data[data.columns[-1]]}))
# print({elem for elem in update_class(data[data.columns[-1]],1,-1)})
# print(type(data['LB']))
# print(len(data['LB']))


data_train = data.sample(frac=0.7, random_state=43)
data_test = data[~data.index.isin(data_train.index)]




X = data.drop(columns='CLASE').to_numpy()
Y = data['CLASE'].to_numpy()


X_train = data_train.drop(columns='CLASE').to_numpy()
Y_train = data_train['CLASE'].to_numpy()

X_test = data_test.drop(columns='CLASE').to_numpy()
Y_test = data_test['CLASE'].to_numpy()




[1, 2, 3]


In [7]:


# k_folds(X,Y,10,multi_logistic(0.15,1000,batch_size=len(Y)))


'''Tiempo total: 211.1885039806366
[0.86592376 0.56470236 0.98333333] [0.98789498 0.33421913 0.35307015] [0.92255581 0.41236083 0.51718765]'''

'Tiempo total: 211.1885039806366\n[0.86592376 0.56470236 0.98333333] [0.98789498 0.33421913 0.35307015] [0.92255581 0.41236083 0.51718765]'

In [8]:
# k_folds(X,Y,10,KNN())


'''
Tiempo total: 2.3603439331054688e-05
[0.94109457 0.74969949 0.89357127] [0.97239557 0.66945089 0.77829994] [0.95640472 0.70640609 0.83059618]
'''

'\nTiempo total: 2.3603439331054688e-05\n[0.94109457 0.74969949 0.89357127] [0.97239557 0.66945089 0.77829994] [0.95640472 0.70640609 0.83059618]\n'

In [9]:

# k_folds(X,Y,10,DT())


'''
Tiempo total: 65.01346898078918
[0.95675166 0.76444418 0.91935713] [0.96083191 0.75820519 0.89218247] [0.95865666 0.7581321  0.90303412]
'''

'\nTiempo total: 65.01346898078918\n[0.95675166 0.76444418 0.91935713] [0.96083191 0.75820519 0.89218247] [0.95865666 0.7581321  0.90303412]\n'

In [10]:

# k_folds(X,Y,10,multi_svm(0.0001,1000,batch_size=200))

'''
Tiempo total: 33.13960003852844
[0.87043871 0.57830617 0.91874237] [0.99423027 0.27269007 0.54927161] [0.92779656 0.36098266 0.67764121]
'''

'\nTiempo total: 33.13960003852844\n[0.87043871 0.57830617 0.91874237] [0.99423027 0.27269007 0.54927161] [0.92779656 0.36098266 0.67764121]\n'

In [11]:

k_folds(X,Y,10,multi_svm(0.0001,1000,batch_size=len(Y)))

0
Para el fold 0 se demoro 21.628988027572632 para entrenar
Tiempo total: 21.628988027572632
0.0 0.0 0.0


In [12]:
# k_folds(X,Y,10,multi_logistic(0.15,1000,batch_size=200))


'''
Tiempo total: 23.796767473220825
[0.85347027 0.50932012 0.97424242] [0.99495131 0.24096558 0.38434939] [0.91837534 0.32078181 0.53575505
'''

'\nTiempo total: 23.796767473220825\n[0.85347027 0.50932012 0.97424242] [0.99495131 0.24096558 0.38434939] [0.91837534 0.32078181 0.53575505\n'

In [13]:
# print(X_train.shape)
# print(Y_test.shape)

In [14]:
#initial_verification 
# alpha = 0.0001
# epochs = 1000
# c = 10
# clases = list({elem for elem in data[data.columns[-1]]})

# model = multi_svm(alpha,epochs,c,batch_size=len(Y))
# # model = multi_logistic(alpha,epochs)
# model.train(X_train,Y_train,X_test,Y_test,clases)


# # model = DT(X_train,Y_train)







In [15]:

# clases = list({elem for elem in Y})
# clases_map = {clases[i]:i for i in range(len(clases))}

# pred = model.predict(X_test)


# one_hot_data_pred = to_multi_label(pred,clases_map)

# one_hot_real_data = to_multi_label(Y_test,clases_map)


# print(confusion_matrix(Y_test,pred))